# Add many "Section discharge" result specifications

This example shows how to add multiple "Section discharge" result specifications for 2D overland models. Basic workflow is:

1. Import section coordinates.
2. Create result specification for each section.
3. Add result specifications to a particular simulation ID.

#### Setup a dummy database to work with

In [ ]:
from test_utils import create_sqlite_db
db_path = "my_model.sqlite"
create_sqlite_db(db_path)

#### Required imports

In [ ]:
from mikeplus import DataTableAccess
from mikeplus.fieldTableNames.tableNames import CSTabNames
from mikeplus.fieldTableNames.fieldNames import Fields
from System import Nullable

# Note: GeoPandas and Contextily is not installed by default with MIKE+Py
import geopandas as gpd
import contextily as cx

#### 1. Import section coordinates.

In [ ]:
# Read sections from shapefile into GeoDataFrame
gdf = gpd.read_file("../tests/testdata/notebooks/sections.shp")
gdf

In [ ]:
# Plot the sections
ax = gdf.plot(column='name', legend=True)
cx.add_basemap(ax, crs=gdf.crs, source=cx.providers.CartoDB.Positron)

In [ ]:
# Create a dictionary with keys as section names and values as a list of xy coordinates of the section
sections_xy = {}
for name in gdf['name']:
    sections_xy[name] = list(gdf[gdf['name'] == name].geometry.values[0].coords)
print("Section A", sections_xy['Section A'])

#### 2. Create result specification for each section.

In [ ]:
# Open the model database
dta = DataTableAccess(db_path)
dta.open_database()
dta

In [ ]:
# Create one result specification per cross section with the appropriate types
for name in sections_xy:
    dta.insert(CSTabNames.RSTable, muid=name, values={
    Fields.RSFields.ModelTypeNo: Nullable[int](3),    # 3 = 2D Overland
    Fields.RSFields.ContentTypeNo: Nullable[int](21), # 21 = Section discharge
    Fields.RSFields.FormatNo: Nullable[int](2),       # 2 = Dfs0
})

In [ ]:
# Insert the secion coordinates into the database. Note that coordinates must be in the same projection as the model.
table = dta.datatables[CSTabNames.RSSGeomTable]
for name, xy in sections_xy.items():
    selection_id = dta.get_muid_where(CSTabNames.RSSTable, f"{Fields.RSSFields.ResultSpecID}=='{name}'")[0]
    for i, (x, y) in enumerate(xy):
        muid = table.CreateUniqueMuid()
        row = table.CreateRowData(muid)
        table.Insert(row)
        dta.set_values(CSTabNames.RSSGeomTable, muid, {Fields.RSSGeomFields.X:x, Fields.RSSGeomFields.Y:y, Fields.SqnFieldBase.Sqn:Nullable[int](i+1), Fields.RSSGeomFields.SelectionID:selection_id})

#### 3. Add result specifications to a particular simulation ID.

In [ ]:
# Create a new 2D overland simulation
simulation_id = "my_simulation"

row = dta.datatables[CSTabNames.ProjectTable].CreateRowData(simulation_id)
dta.datatables[CSTabNames.ProjectTable].Insert(row)

dta.set_values(CSTabNames.ProjectTable, simulation_id, {
    Fields.ProjectFields.Enable_2DOverland: Nullable[int](1),
    Fields.ProjectFields.Enable_HD: Nullable[int](1),
})

In [ ]:
# Add all of the new result specifications to the simulation
for name in sections_xy:
    
    output_id = dta.datatables[CSTabNames.ProjectOutputTable].CreateUniqueMuid()
    row = dta.datatables[CSTabNames.ProjectOutputTable].CreateRowData(output_id)
    dta.datatables[CSTabNames.ProjectOutputTable].Insert(row)

    dta.set_values(CSTabNames.ProjectOutputTable, output_id, {
        Fields.ProjectOutputFields.OutputID: name,
        Fields.ProjectOutputFields.SimulationID: simulation_id,
        Fields.ProjectOutputFields.ContentsTypeNo: Nullable[int](21), # 21 = Section discharge
        Fields.ProjectOutputFields.FormatNo: Nullable[int](2),        # 2 = Dfs0
    })

In [ ]:
# Close the database
dta.close_database()

#### Clean up

In [ ]:
from pathlib import Path
db_path = Path(db_path)
for file in [db_path, db_path.with_suffix(".mupp")]:
    if file.exists():
        file.unlink()